In [4]:
from IPython.display import display
import dash
import sys,os
if  not os.path.abspath('./') in sys.path:
    sys.path.append(os.path.abspath('./'))
if  not os.path.abspath('../') in sys.path:
    sys.path.append(os.path.abspath('../'))
if  not os.path.abspath('../../dashgrid') in sys.path:
    sys.path.append(os.path.abspath('../../dashgrid'))
if  not os.path.abspath('../../dashgrid/dashgrid') in sys.path:
    sys.path.append(os.path.abspath('../../dashgrid/dashgrid'))

import datetime
import pandas as pd
import numpy as np
from dashgrid import dgrid_components as dgc
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)


In [5]:
all_days=1000
end_date = datetime.datetime.now()
beg_date = end_date - datetime.timedelta(all_days)
date_series = pd.bdate_range(beg_date,end_date)
trade_dates = date_series.astype(str).str.replace('-','').astype(int)
n = len(trade_dates)
changes = np.random.lognormal(0,.15/256**.5,n-1)
initial = np.array([100.0])
closes = np.cumprod(np.append(initial,changes)).round(2)
open_ranges = np.random.lognormal(0,.3/256**.5,n)
opens = (closes * open_ranges).round(2)
low_ranges = np.random.lognormal(.1,.2/256**.5,n)
lows = np.array([min(x,y) for x,y in zip(opens,closes)]) - low_ranges
lows = lows.round(2)
high_ranges = np.random.lognormal(.1,.2/256**.5,n)
highs = np.array([max(x,y) for x,y in zip(opens,closes)]) + high_ranges
highs = highs.round(2)

volume_changes = np.random.lognormal(0,.50/256**.5,n-1)
initial_volume = np.array([1000000.0]) * np.random.lognormal(0,.15/256**.5)
volumes = np.cumprod(np.append(initial_volume,volume_changes)).round(0)


df_pseudo = dgc.make_df({'date':trade_dates,'open':opens,'high':highs,'low':lows,
                  'close':closes,'volume':volumes})


In [6]:
fig1 = dgc.PlotlyCandles(df_pseudo.iloc[-50:],number_of_ticks_display=20,title='Random Walk')
iplot(fig1.get_figure())

In [7]:
no_border={
    'border': '1px solid #000',
    'grid-gap': '10px 10px',
}
df_save_list = []
logger = dgc.li.init_root_logger('logfile.log','INFO')

def dash_app_1():
    days_to_show = 50
    df_to_show = df_pseudo.iloc[-days_to_show:]
    
    dt1_comp = dgc.DashTableComponent(
        'dt1',df_to_show,None,title='Random Walk',
        editable_columns=['close'],logger = logger
    )
    def create_figure_from_df(input_list):
        if input_list is None or len(input_list)<1 or input_list[0] is None:
            return [None]
        dict_df = input_list[0]
        df = dgc.make_df(dict_df)
        df.close = df.close.astype(str).astype(float)
        p = dgc.PlotlyCandles(df,number_of_ticks_display=20,title='Random Walk').get_figure()
        return p
    
    gr1_comp = dgc.FigureComponent('f1',dt1_comp,create_figure_from_df,logger=logger)

    app = dash.Dash()
    comp_list = [dt1_comp,gr1_comp]
    html_list = [dgc.create_grid([c],num_columns=1) for c in comp_list]
    app.layout = dgc.html.Div(html_list)
    [c.callback(app) for c in comp_list]
    return app


In [8]:
DASH_APP_TO_RUN = dash_app_1
app = DASH_APP_TO_RUN()
app.run_server(host='127.0.0.1',port=8500)

dt.columns [{'name': 'date', 'id': 'date', 'editable': False}, {'name': 'open', 'id': 'open', 'editable': False}, {'name': 'high', 'id': 'high', 'editable': False}, {'name': 'low', 'id': 'low', 'editable': False}, {'name': 'close', 'id': 'close', 'editable': True}, {'name': 'volume', 'id': 'volume', 'editable': False}]
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2019-07-22 22:40:00,175 - werkzeug - INFO -  * Running on http://127.0.0.1:8500/ (Press CTRL+C to quit)
2019-07-22 22:40:04,909 - werkzeug - INFO - 127.0.0.1 - - [22/Jul/2019 22:40:04] "GET / HTTP/1.1" 200 -
2019-07-22 22:40:04,959 - werkzeug - INFO - 127.0.0.1 - - [22/Jul/2019 22:40:04] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1563025507 HTTP/1.1" 200 -
2019-07-22 22:40:04,967 - werkzeug - INFO - 127.0.0.1 - - [22/Jul/2019 22:40:04] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1563025507 HTTP/1.1" 200 -
2019-07-22 22:40:04,980 - werkzeug - INFO - 127.0.0.1 - - [22/Jul/2019 22:40:04] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1563025507 HTTP/1.1" 200 -
2019-07-22 22:40:05,006 - werkzeug - INFO - 127.0.0.1 - - [22/Jul/2019 22:40:05] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.0.0&m=1563025508 HTTP/1.1" 200 -
2019-07-22 22:40:05,026 - werkzeug - INFO - 127.0.0.1 

In [9]:
l = dgc.flatten_layout(app)
for c in l:
    print('***********************************************')
    print(c)

***********************************************
DataTable(columns=[{'name': 'date', 'id': 'date', 'editable': False}, {'name': 'open', 'id': 'open', 'editable': False}, {'name': 'high', 'id': 'high', 'editable': False}, {'name': 'low', 'id': 'low', 'editable': False}, {'name': 'close', 'id': 'close', 'editable': True}, {'name': 'volume', 'id': 'volume', 'editable': False}], css=[{'selector': 'table', 'rule': 'width: 100%;'}], data=[{'date': 20190514.0, 'open': 109.21, 'high': 111.56, 'low': 108.09, 'close': 110.46, 'volume': 1412272.0}, {'date': 20190515.0, 'open': 109.09, 'high': 112.41, 'low': 107.98, 'close': 111.3, 'volume': 1396075.0}, {'date': 20190516.0, 'open': 110.51, 'high': 113.64, 'low': 109.4, 'close': 112.54, 'volume': 1386180.0}, {'date': 20190517.0, 'open': 113.54, 'high': 114.62, 'low': 111.22, 'close': 112.33, 'volume': 1369202.0}, {'date': 20190520.0, 'open': 112.83, 'high': 113.93, 'low': 110.73, 'close': 111.83, 'volume': 1327937.0}, {'date': 20190521.0, 'open': 10

In [10]:
import importlib
importlib.reload(dgc)

<module 'dashgrid.dgrid_components' from '/Users/bperlman1/Documents/billybyte/pyliverisk/dashgrid/dashgrid/dgrid_components.py'>

In [11]:
df1 = df_save_list[0]

IndexError: list index out of range

In [ ]:
df1.to_dict('rows')